In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
autentica = pd.read_excel('ls.xlsx')
login = str(autentica['login'][0])
senha = str(autentica['senha'][0])

# display(autentica)

pedidos = pd.read_excel('consolidacao/pedidos/CAFI_pedidos.xlsx')
pedidos = pedidos[pedidos['acao'].str.contains('pedido no suap')]

# separando elementos de despesa
pedidos_ed = pedidos['ed'].drop_duplicates()
pedidos_ed.index = [x for x in range(0, len(pedidos_ed))]
# print(pedidos_ed)

# pedidos = pedidos[pedidos['ed'].str.contains(str(pedidos_ed[12]))]


# reindexando os pedidos
pedidos.index = [x for x in range(1, len(pedidos)+1)]
pedidos['i'] = [x for x in range(1, len(pedidos)+1)]
# pedidos = pedidos.iloc[0:6]
# display(pedidos)
# display(pedidos.info())
qtd = pedidos[['i', 'diferenca']]
dict_qtd = qtd.to_dict('records')
ped = pedidos[['i', 'codigo']]
dict_ped = ped.to_dict('records')

indisponivel = "indisponivel"

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
import time

def abrir_suap():
    
    global nav
    nav = webdriver.Chrome()
    nav.get("https://suap.ifpe.edu.br/accounts/login/?next=/")
    # inserindo usuário
    nav.find_element_by_xpath('//*[@id="id_username"]').click()
    nav.find_element_by_xpath('//*[@id="id_username"]').send_keys(login)

    # inserindo senha 
    nav.find_element_by_xpath('//*[@id="id_password"]').click()
    nav.find_element_by_xpath('//*[@id="id_password"]').send_keys(senha)

    nav.find_element_by_xpath('//*[@id="ok"]').click()

In [4]:
df_elemento_despesa = pd.DataFrame()
df_temp = pd.DataFrame(columns=('value', 'ed'))
df_temp.loc[0] = ['14', 'esportes']
# display(df_temp)

opcoes = []
valores = []

def varrer_ed():
#     abrir_suap()
    nav.get("https://suap.ifpe.edu.br/almoxarifado/form_busca/materialconsumo/")
        
    valor_ed = nav.find_element_by_id('id_elemento_despesa')
    options = [x for x in valor_ed.find_elements_by_tag_name('option')]
    
    for i in options:
        # print(i.get_attribute('value')+" "+i.text)
        opcoes.append(i.text)
        valores.append(i.get_attribute('value'))
#         print(f'{i.text}-{i.get_attribute("value")}')

    df_elemento_despesa['value'] = valores
    df_elemento_despesa['ed']= opcoes
    
#     display(df_elemento_despesa)

In [5]:
def get_qtd():
    
    abrir_suap()
    
    df_tabela = pd.DataFrame()
    tabela = []
    codigos = []
    unidades = []
    descricoes = []
    quantidades = []
    valores = []
    elemento = []
    
    try:
        pag = 1
        saida = True
        while saida:
                
            nav.get(f'https://suap.ifpe.edu.br/almoxarifado/form_busca/materialconsumo/?page={pag}&material=&elemento_despesa=&estoque_tipo=emcampus&unidade_medida=')
            celula = nav.find_element_by_tag_name('td')
                
            if celula:
                lista = nav.find_element_by_tag_name('table')
                td = lista.find_elements_by_tag_name('td')
                
                for i in td:
                    tabela.append(i.text)
                
                pag += 1
            else:
                saida = False
                
    except NoSuchElementException:
        print('acabaram as páginas')
        
    for cod in range(0, len(tabela), 7):
        codigos.append(tabela[cod])

    for des in range(1, len(tabela), 7):
        descricoes.append(tabela[des])
        
    for uni in range(2, len(tabela), 7):
        unidades.append(tabela[uni])

    for qtd in range(4, len(tabela), 7):
        quantidades.append(tabela[qtd])
    
    for val in range(5, len(tabela), 7):
        valores.append(tabela[val])
        
    for ed in range(3, len(tabela), 7):
        elemento.append(tabela[ed])

    df_tabela['codigo'] = codigos
    df_tabela['descricao'] = descricoes
    df_tabela['und'] = unidades
    
    df_tabela['qtd'] = quantidades
    df_tabela['qtd'] = pd.to_numeric(df_tabela['qtd'])
    
    df_tabela['vlr_total'] = valores
    df_tabela['vlr_total'] = df_tabela['vlr_total'].str[2:]
    df_tabela['vlr_total'] = pd.to_numeric(df_tabela['vlr_total'])
    
    df_tabela['vlr_unt'] = df_tabela['vlr_total'] / df_tabela['qtd']
    df_tabela['vlr_unt'] = df_tabela['vlr_unt'].round(2)
    
    df_tabela['ed'] = elemento
    
    df_tabela.to_excel('relatorio/relatorio.xlsx', index=False)
    display(df_tabela)
    
    nav.quit()


In [6]:
def set_pedidos():
    abrir_suap()

    nav.get('https://suap.ifpe.edu.br/almoxarifado/form_requisicao_pessoa_pedido/')

    # preenchendo campo do requisitante
    nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/fieldset/div[2]/input[1]').send_keys('Andrea Dacal')
    time.sleep(3)
    nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/fieldset/div[2]/input[1]').send_keys(Keys.DOWN)
    nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/fieldset/div[2]/input[1]').send_keys(Keys.ENTER)

    # preenchento campo do pedido do material

    for i in dict_ped:
        nav.find_element_by_xpath(f'/html/body/div[1]/div[3]/form/div[1]/fieldset/div[{i["i"]}]/input[1]').send_keys(i['codigo'])
        time.sleep(3)
        nav.find_element_by_xpath(f'/html/body/div[1]/div[3]/form/div[1]/fieldset/div[{i["i"]}]/input[1]').send_keys(Keys.DOWN)
        nav.find_element_by_xpath(f'/html/body/div[1]/div[3]/form/div[1]/fieldset/div[{i["i"]}]/input[1]').send_keys(Keys.ENTER)
        time.sleep(2)
        nav.find_element_by_name('addItem').click()

    for i in dict_qtd:    
        nav.find_element_by_xpath(f'/html/body/div[1]/div[3]/form/div[1]/fieldset/div[{i["i"]}]/input[3]').send_keys(i['diferenca'])

In [7]:
get_qtd()

acabaram as páginas


,codigo,descricao,und,qtd,vlr_total,vlr_unt,ed
0,30.07.728,"ÁGUA MINERAL ENVASADA, GARRAFÃO DE 20 LITROS.",UN,130,884.00,6.80,07 - GENEROS DE ALIMENTACAO
1,30.14.117,"APITO, MATERIAL PLÁSTICO, APLICAÇÃO ESPORTE, T...",UN,14,169.86,12.13,14 - MATERIAL EDUCATIVO E ESPORTIVO
2,30.14.120,BOLA TENIS DE MESA,UN,12,23.88,1.99,14 - MATERIAL EDUCATIVO E ESPORTIVO
3,30.14.121,REDE ESPORTE BASQUETE,UN,7,111.93,15.99,14 - MATERIAL EDUCATIVO E ESPORTIVO
4,30.14.122,REDE ESPORTE FUTSAL,UN,10,1099.40,109.94,14 - MATERIAL EDUCATIVO E ESPORTIVO
...,...,...,...,...,...,...,...
588,30.42.982,"BROCA DE VIDEA, MATERIAL AÇO RÁPIDO, 10 MM, CO...",UN,5,34.55,6.91,42 - FERRAMENTAS
589,30.42.995,"BROCA DE WÍDIA, 6 MM.",UN,5,17.60,3.52,42 - FERRAMENTAS
590,30.44.67,"Fita adesiva, material propileno, 20mm x 30m, ...",ROLO,10,223.00,22.30,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS
591,30.44.69,PLACA DE SINALIZAÇÃO DE AMBIENTE,UN,2,43.00,21.50,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS
